### Phonotatic Constraits Analysis

Atempt of morphological decipherement (via phonotatic constraits) based on the hypothesis of Naibbe Cipher: https://youtu.be/ByARtG-GUPo?t=5559

In [5]:
import re
import pandas as pd

class VoynichTextProcessor:
    """Processes Voynich Manuscript text to return a cleaned DataFrame."""
    
    def __init__(self):
        self.raw_text = None

    def load_raw_text(self, filepath: str) -> bool:
        """Load raw text from file."""
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                self.raw_text = f.read()
            return True
        except FileNotFoundError:
            return False

    def clean_voynich_text(self, filepath: str, treat_commas_as_spaces: bool = True, min_word_length: int = 2) -> pd.DataFrame:
        """Clean Voynich Manuscript text and return a DataFrame"""
        if not self.load_raw_text(filepath):
            return pd.DataFrame(columns=['folio', 'text'])

        lines = self.raw_text.strip().split('\n')
        folio_pattern = r'<f(\d+)([rv])?\.'
        data = []
        current_folio = None

        def replace_uncertain(match):
            options = match.group(1).split(':')
            return options[0] if options else ''

        for line in lines:
            line = line.strip()
            if not line or line.startswith('#'):
                continue

            cleaned_line = line
            cleaned_line = re.sub(r'@\d+', '', cleaned_line)  # Remove annotations
            cleaned_line = re.sub(r'<![^>]*>', '', cleaned_line)  # Remove comments
            cleaned_line = re.sub(r'<[^>]*>', '', cleaned_line)  # Remove other markup
            cleaned_line = re.sub(r'\[([^\]]+)\]', replace_uncertain, cleaned_line)  # Handle uncertain readings
            cleaned_line = re.sub(r'[{}]', '', cleaned_line)  # Remove braces
            cleaned_line = re.sub(r'\?+', '', cleaned_line)  # Remove question marks
            cleaned_line = re.sub(r'[^a-zA-Z\s,.]', '', cleaned_line)  # Keep only letters, spaces, commas, periods
            if treat_commas_as_spaces:
                cleaned_line = cleaned_line.replace('.', ' ').replace(',', ' ')
            else:
                cleaned_line = cleaned_line.replace('.', ' ').replace(',', '')
            cleaned_line = re.sub(r'\s+', ' ', cleaned_line).strip().lower()  # Normalize spaces, lowercase

            folio_match = re.search(folio_pattern, line)
            if folio_match:
                current_folio = f"{folio_match.group(1)}{folio_match.group(2) or 'r'}"
            elif re.match(folio_pattern, cleaned_line):
                continue

            if cleaned_line and current_folio:
                words = cleaned_line.split()
                clean_words = [w for w in words if re.match(r'^[a-z]+$', w) and len(w) >= min_word_length]
                if clean_words:
                    data.append({'folio': current_folio, 'text': ' '.join(clean_words)})

        text = pd.DataFrame(data, columns=['folio', 'text'])
        return text

if __name__ == "__main__":
    processor = VoynichTextProcessor()
    filepath = "transliteration_zl.txt"
    text = processor.clean_voynich_text(filepath)

text

,folio,text
0,1r,fachys ykal ar ataiin shol shory cthres kor sh...
1,1r,sory ckhar or kair chtaiin shar ase cthar ctha...
2,1r,syaiir sheky or ykaiin shod cthoary cthes dara...
3,1r,soiin oteey oteos roloty cthiar daiin okaiin o...
4,1r,sair chear cthaiin cphar cfhaiin
...,...,...
5302,116r,osain shky qorain chckhey qokey lkechy okeey o...
5303,116r,sykar ain olkeey dainchey qokar chey dain otan...
5304,116r,sysor shey qokey okeolan chey qol or cheey qor...
5305,116r,sodal ch al chcthy chckhy qol ain ary


In [2]:
import re
import pandas as pd
from collections import Counter

def get_folio_number(folio):
    """Extract numeric part of folio (e.g., '1r' -> 1)."""
    match = re.match(r'(\d+)', folio)
    return int(match.group(1)) if match else 0

def generate_ngrams(words, n):
    """Generate n-grams from a list of words."""
    return [' '.join(words[i:i+n]) for i in range(len(words)-n+1)]

def find_ngram_frequencies(text_df, max_n=10):
    """Return a DataFrame with longest n-grams (2 to max_n) with frequencies >1 for folios 1 to 60."""
    # Filter for folios 1 to 60
    filtered_text = text_df[text_df['folio'].apply(get_folio_number) <= 55]
    
    # Concatenate all text into one list of words
    all_words = ' '.join(filtered_text['text'].fillna('').tolist()).split()
    
    # Collect n-gram data
    ngram_data = []
    
    # Generate and count n-grams for n=2 to max_n
    for n in range(2, max_n + 1):
        ngrams = generate_ngrams(all_words, n)
        ngram_counts = Counter(ngrams)
        # Only include n-grams with count > 1
        for seq, count in ngram_counts.items():
            if count > 1:
                ngram_data.append({'n': n, 'sequence': seq, 'count': count})
    
    # Create DataFrame
    df = pd.DataFrame(ngram_data, columns=['n', 'sequence', 'count'])
    
    if df.empty:
        return df
    
    # Filter out shorter n-grams contained in longer ones
    longest_ngrams = []
    for n in range(max_n, 1, -1):  # Start with longest n-grams
        current_ngrams = df[df['n'] == n]
        for _, row in current_ngrams.iterrows():
            seq = row['sequence']
            is_subsequence = False
            # Check if this sequence is contained in any longer n-gram
            for _, longer_row in df[df['n'] > n].iterrows():
                if seq in longer_row['sequence']:
                    is_subsequence = True
                    break
            if not is_subsequence:
                longest_ngrams.append(row)
    
    # Create final DataFrame with only longest n-grams
    result_df = pd.DataFrame(longest_ngrams, columns=['n', 'sequence', 'count'])
    return result_df.sort_values(by=['n', 'count'], ascending=[False, True])

ngram_df = find_ngram_frequencies(text, max_n=10)
ngram_df

,n,sequence,count
348,3,chol chol kor,2
349,3,chol daiin cthy,2
350,3,daiin cthy schey,2
351,3,chol chy chaiin,2
352,3,cthor chol chor,2
...,...,...,...
84,2,shol daiin,7
88,2,otol chol,7
102,2,chor daiin,8
161,2,daiin dain,9


In [14]:
import pandas as pd
from collections import Counter, defaultdict
import re

def map_valid_sequences(text_df, transitions_df, max_n=10):
    def get_folio_number(folio):
        match = re.match(r'(\d+)', folio)
        return int(match.group(1)) if match else 0
    filtered_text = text_df[text_df['folio'].apply(get_folio_number) <= 60]
    
    successor_dict = {row['symbol']: set(row['successors'].split(', ')) if row['successors'] else set() for _, row in transitions_df.iterrows()}
    
    sequence_folios = defaultdict(list)
    sequence_counts = Counter()
    
    for _, row in filtered_text.iterrows():
        folio = row['folio']
        words = row['text'].split()
        if len(words) < 2:
            continue
        for n in range(2, min(max_n + 1, len(words) + 1)):
            for i in range(len(words) - n + 1):
                subsequence = words[i:i+n]
                is_valid = True
                for j in range(len(subsequence) - 1):
                    current = subsequence[j]
                    next_word = subsequence[j + 1]
                    if current not in successor_dict or next_word not in successor_dict[current]:
                        is_valid = False
                        break
                if is_valid:
                    seq_str = ' '.join(subsequence)
                    sequence_counts[seq_str] += 1
                    if folio not in sequence_folios[seq_str]:
                        sequence_folios[seq_str].append(folio)
    
    valid_sequences = []
    for seq, folios in sequence_folios.items():
        valid_sequences.append({
            'folio': ', '.join(sorted(folios)),
            'sequence': seq,
            'length': len(seq.split()),
            'frequency': sequence_counts[seq]
        })
    
    df = pd.DataFrame(valid_sequences)
    if df.empty:
        return df
    return df.sort_values(by=['frequency','length'], ascending=[False, False])

# Run the function
valid_sequences_df = map_valid_sequences(text, transitions_df)
valid_sequences_df

,folio,sequence,length,frequency
33,"10v, 14v, 15r, 15v, 16v, 19v, 1v, 20v, 21r, 21...",chol daiin,2,26
10,"15v, 17v, 1r, 29v, 42r, 44v, 45r, 47r, 47v, 54...",chol chol,2,18
190,"10r, 20v, 31v, 33r, 33v, 39v, 40r, 43v, 55r, 58v",or aiin,2,14
63,"11r, 15v, 22v, 32r, 3r, 44v, 45v, 51v, 9v",daiin cthy,2,9
119,"15v, 20v, 22v, 25v, 42r, 47r, 56v, 5v",shol daiin,2,8
...,...,...,...,...
680,54v,dar chor,2,1
688,55r,daiin sho,2,1
690,55r,daiin otol,2,1
691,55v,aiin daiin,2,1
